# Linear Regression using Scikit-Learn

In this example, we use:
- `scikit-learn` library to train a linear regression model
- `seaborn` and `matplotlib` libraries to visualize the data and the model
- California Housing dataset from `scikit-learn` library. 

In [ ]:
import numpy as np
import pandas as pd

As we discussed in [ML Workflow](../ml_workflow.md), there are key steps before we are ready for modelling. Those steps are data engineering, EPA (Exploratory Data Analysis), and feature engineering. In this simple example, we touch on those steps breifly.

## Data Engineering

In this example, our training dataset is reading to use from the `sklearn.datasets` module.

### Load the data
In this example, the data engineering steps (collect, cleanse and preprocess the data) are already done for us. So, we simply use the California Housing dataset from `sklearn.datasets` module. This dataset has around 20,000 samples and 8 features.

In [ ]:
# Using built-in dataset from sklearn
from sklearn.datasets import fetch_california_housing

# Load the California Housing dataset.
housing_data = fetch_california_housing(as_frame=True)

In [ ]:
print(housing_data.DESCR)

## Exploratory Data Analysis (EDA) and Feature Engineering

We will briefly [explore and visualize](../ml_workflow.md#visualization-techniques)   the data and their relationship. Also, we will briefly touch on common steps of _Feature Selection_, _Feature Creation_ and _Feature Scaling_. 

Let's start getting familiar with our data.  The goal of this step is to first understand our data, and then to engineer the features in a way that our dataset is ready for training a high performing model. 


### Shape and Types of data

Let's start with the type and shapes of our dataset.

In [ ]:
# X is the input features.
X = housing_data.data

# y is the target/label
y = housing_data.target

print(f"X type: {type(X)}, X shape: {X.shape}")
print(f"y type: {type(y)}, y shape: {y.shape}")

`X` is a matrix of `m` samples and `n` features. Here `m = 20640` and `n = 8`. So, we have 8 features and total of 20640 samples.

`y` is a vector of `m` samples. So, since here we are doing a _superivsed learning_ task, we should have target values for each of the samples. So, `y` is a vector of 20640 values.

In [ ]:
print("X values:")
print(X.head())

print("\ny values in $100,000:")
print(y.head())

We can inspect each of the examples individually by using `pandas` features.

In [ ]:
# The first row of X and its target y values:
print("\nFirst row of X and it's target value y:")
print("X_1:")
print(X.iloc[0])
print("y_1:")
print(y[0])

### Detecting Outliers
Now let's explore our data to check for any outliers. Detecting outliers is important as they can have a disproportionate effect on the model and lean to poor performance.


In [ ]:
# Range of each feature
print(X.describe())

In the above, we already can see some outliers. For example, the `AveRooms` feature has some very high values e.g. 141 which is probably an outlier for a typical housing dataset. Or the number of occupants per household `AveOccup` has some very high values e.g. 1243. Those are probably outliers.

However before we decide to remove them, let's visualize the data and see if we can get more insights. We'll use `seaborn` and `matplotlib` libraries for visualization. 

> Note: EDA and Feature Engineering is a process that needs to be done carefully and iteratively. This process needs a lot of domain knowledge and experience. For example, a valid but rare case might be considered as an outlier, etc.  


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

**Box Plot**  
A box plot is a standardized way of displaying the distribution of data based on a five-number summary: minimum, first quartile (Q1), median, third quartile (Q3), and maximum. It can tell you about your outliers and what their values are. It can also tell you if your data is symmetrical, how tightly your data is grouped, and if and how your data is skewed.



In the following we plot the box plot for one of the features `AveRooms`. Depending on the domain and the data, we can decide which features to start with for the clean up process. 

In [ ]:
sns.boxplot(data=X["AveRooms"])

In the boxplot:
- **Top Line (Upper Whisker)**: The top line represents the Q3, which the value below which 75% of the data falls.
- **Middle Line**: The middle box also called **Interquartile Range (IQR)** which represents the median, which is the middle value of the dataset. $IQR = Q3 - Q1$
- **Bottom Line (Lower Whisker)**: The bottom line represents the Q1, which the value below which 25% of the data falls. 

**Beyond the Whiskers**: The small circles are the indicator of potential outliers, as they fall beyond the acceptable range of typical data variability.


#### Clean up the outliers
We can start by determining the range of acceptable values using the **IQR (Interquartile Range)** method, which is standard for identifying outliers. However, we can apply a **softer approach** by increasing the threshold multiplier (e.g., $k = 2.5$ instead of the usual $1.5$) to only exclude extreme outliers. 


1. **Compute IQR:**
   - Calculate $Q1$ (25th percentile) and $Q3$ (75th percentile) of `AveRooms`.
   - Compute the IQR: $ \text{IQR} = Q3 - Q1 $.
   - Define thresholds:
    $$ \text{Lower Bound} = Q1 - k \times \text{IQR} $$
    $$ \text{Upper Bound} = Q3 + k \times \text{IQR} $$
     We'll start with a **softer multiplier** like $k = 2.5$.


In [ ]:
# Compute Q1, Q3, and IQR
Q1 = X["AveRooms"].quantile(0.25)
Q3 = X["AveRooms"].quantile(0.75)
IQR = Q3 - Q1

# Define bounds (using a softer threshold with k = 2.5)
k = 2.5
lower_bound = Q1 - k * IQR
upper_bound = Q3 + k * IQR

print(f"Q1: {Q1}, Q3: {Q3}, IQR: {IQR}")
print(f"Lower bound: {lower_bound}")
print(f"Upper bound: {upper_bound}")


2. **Filter Out Outliers:**
   - Remove rows where the `AveRooms` value is outside the bounds.


In [ ]:
# Step 3: Filter the dataset

# Let's make a copy of the original dataset
original_X = X.copy()
original_y = y.copy()

# Filter the dataset based on the bounds
X = X[(X["AveRooms"] >= lower_bound) & (X["AveRooms"] <= upper_bound)]

# Filter the target variable based on the filtered features
y = y[X.index]

print(f"Original dataset size: {original_X.shape[0]}")
print(f"Cleaned dataset size: {X.shape[0]}")

The cleaned up removed a tiny fraction of the data, which is a good sign that the data is not too noisy, and give us confidence that we probably removed the outliers, not the valid data.

3. **Validate:**
   - Visualize the boxplot again to ensure the outliers are handled properly.

In [ ]:
sns.boxplot(data=X["AveRooms"])

Alright, this is a better plot now. We can see the obvious outliers are removed. You can continue this by adjusting the threshold of $k$ to lower or higher values and see how it affects the data.

We can repeat the same process for other features as well and clean up the outliers. Although, we may decide to have a light clean up first, then go through the next steps which help us decide about the feature selection. Then after knowing the chosen features, we can come back to this step and do a more thorough clean up.

> Note: EDA and Feature Engineering (similar to the whole ML process) is an **empirical process** and needs to be done **iteratively**. We may need to come back to this step after the feature selection and engineering steps. 

### Correlation Heatmap

This is a useful way to visualize the relationship between features (and features-target) which can help us in feature selection. This view in particular is useful to detect multicollinearity (when two or more features are highly correlated, i.e move of one, impact the other).

**Correlation**    
 means that those two features are moving together. Correlation values range from -1 to 1. For example:
- Positive correlation is when one feature increases, the other feature also increases.
- Negative correlation is when one feature increases, the other feature decreases.

As this influence from a feature to another increases, the correlation value moves towards 1 or -1. For example, two features with a correlation value of $0.7$ are much more correlated than two features with a correlation value of $0.2$. Similarly, two features with a correlation value of $-0.7$ are much more negatively correlated than two features with a correlation value of $-0.2$.



**Multi-collinearity Detection**  
A common use of heatmap is to detect **multi-collinearity** between features. Multi-collinearity is a phenomenon in which two or more features in a dataset are highly correlated. This can cause problems in the model, such as unstable coefficients, and it can make it difficult to determine the effect of each feature on the target variable. In which those cases, we either:
- Remove one of the features (Feature Selection).
- Combine the features and create a new feature from them (New Feature Creation)



In [ ]:
def plot_correlation_heatmap(X, y):
    data_combined = X.copy()
    data_combined["MedHouseVal"] = y
    corr = data_combined.corr()
    plt.figure(figsize=(10, 8))
    plt.title("Correlation Heatmap")
    sns.heatmap(corr, annot=True, cmap="coolwarm", fmt=".2f")
    plt.show()


plot_correlation_heatmap(X, y)

We can use the following guide for interpreting correlation thresholds: 

- Negligible: $|r| < 0.1$ (often ignored)
- Weak: $0.1 \leq |r| < 0.3$
- Moderate: $0.3 \leq |r| < 0.5$
- Strong: $0.5 \leq |r| < 0.7$
- Very Strong: $|r| \geq 0.7$

For feature selection, prioritize $|r| \geq 0.3$, and focus on $|r| \geq 0.5$ for strong relationships unless domain knowledge dictates otherwise.

> The above is not a strict rule, but a guideline which can be adjusted based on the domain knowledge and the data.

Let's Analyze this heatmap: 

**1. Correlation Between Features and the Target (MedHouseVal)**

- **Strong Correlation**: 
  - **MedInc (Median Income)** has a strong positive correlation with **MedHouseVal** (**0.69**). 
    - This means that as median income increases, house values tend to increase as well.
    - This aligns with the economic principle that wealthier areas typically have higher property values.

- **Moderate Correlations**:
  - **AveRooms (Average Rooms per Dwelling)** has a moderate positive correlation (**0.33**) with **MedHouseVal**.
    - More rooms per house might indicate larger or more luxurious properties, which could drive up house prices.

- **Weak Correlations**:
  - **HouseAge (Age of Houses)** shows a weak positive correlation (**0.11**) with **MedHouseVal**.
    - This suggests that older houses might be slightly more valuable in the dataset's context, but the relationship isn't strong.
  - **Latitude** shows a slightly negative correlation (**-0.14**) with **MedHouseVal**, which may indicate a trend where houses further north in the region are slightly less expensive. 
  - **AveBedrms (-0.10)**
- **Negligible Correlations**: 
  - **Population (-0.03)**, **AveOccup (-0.02)**, and **Longitude (-0.05)** show very weak correlations with **MedHouseVal**. These features do not have significant predictive power for house prices on their own.




**2. Multicollinearity Insights**

- **Longitude and Latitude (-0.93)**:
  - This remains a strong negative correlation, as seen before. However, neither **Longitude** nor **Latitude** has a strong individual correlation with the target variable (**MedHouseVal**).
  - This suggests that while geographic location matters, its direct influence on house prices is likely more complex and requires combining these two features (e.g., via clustering or distance to a specific location).

- **AveRooms and AveBedrms (0.23)**:
  - This is a weak positive correlation, meaning these features are loosely related. However, **AveRooms** shows a better relationship with target **MedHouseVal** than **AveBedrms**, so you might prefer to prioritize **AveRooms** in the model.



**Conclusion and Next Steps**

- **Feature Selection**:
    - **MedInc (Median Income)** is clearly the most influential feature for predicting house prices.
    - **AveRooms**, **HouseAge** and **Latitude** might add additional predictive power due to their moderate correlations with the target.
    - So, we'll select `MedInc`, `AveRooms`, and `HouseAge` as our primary features for modeling.

- **Feature Reduction**:
    - Due to the correlation between **AveRooms** and **AveBedrms**, one of these features might be redundant. Since **AveRooms** correlates better with the target, it may be wise to focus on it and drop **AveBedrms**.
    - Features like **Population**, **AveOccup**, **AveBedrms**, and **Longitude** appear to have weaker direct relationships with house prices. So we might consider dropping them to reduce our feature space. 

- **Feature Creation**
    - While neither of geographic features **Latitude** and **Longitude** show strong correlation with house prices individually, their **combined influence** might be significant. For example, a new feature based on the **distance to a central point**, such as the city center or a key landmark.

- **Feature Transformation:** We'll be using normalizaiton technique to scale the features.



In [ ]:
selected_features = ["MedInc", "HouseAge", "AveRooms"]

### Scatter Plot
**Scatter Plot** is a good way to visualize the data. We can show the relationship between two features and a feature with the target variable.


In [ ]:
def plot_feature_target_scatter(X, y):
    data_combined = X.copy()
    data_combined["MedHouseVal"] = y
    sns.pairplot(
        data_combined,
        y_vars="MedHouseVal",
        x_vars=X.columns,
        height=5,
        aspect=0.7,
        kind="scatter",
    )


plot_feature_target_scatter(X, y)

Looking at the scatter plot of each feature against the target variable, we can see the relationship between the features and the target variable. This can help us understand the data better and make decisions about which features to include in the model or what kind of transformation we might need to do on the features.

Looking at the scatter plot confirms our previous analysis regarding the `AveOccup` feature. Not only it has a very weak correlation with the target variable, but also the scatter plot shows that "AveOccup" has a heavy concentration at very low values without a proper distribution, as it lacks a balanced spread across its range and exhibits sparse, extreme outliers. Compare this to other features like `MedInc` and `AveRooms` which have ore uniform distribution across their range.

In [ ]:
def plot_features_target_combined_scatter(X, y):
    data_combined = X.copy()
    # Add the target to the data to plot the combined scatter plot
    data_combined["MedHouseVal"] = y
    sns.pairplot(data_combined)


plot_features_target_combined_scatter(X[selected_features], y)

The above pairplot shows the scatter plot of each feature against the target variable. The diagonal plots are **histograms** of the features. Histograms show the distribution of a feature across its range. For example, the `AveRooms` shows the higher number of houses with around 5 rooms. 




**Insights and the next steps:**
- The plot shows a good spread of data points among the features, and the relationship between the features and the target variable. It also shows a good spread of feature values across their range (in the Histograms). This is a good indication that the selected features are good candidates for the model.
- In the next step, we'll normalize the features to have a similar scale before training the model. Although we have a good spread of data points, the features have different scales. For example, `HouseAge` ranges from 1 to 52, while `AveRooms` ranges from 0.84 to 10. This can cause issues in the model training process, as the model might give more weight (importance) to the features with higher values. So, we'll [normalize](../feature_engineering.md#scaling-and-normalization) the features to have a similar scale before training the model.


In [ ]:
# Range of selected features
print(X[selected_features].describe())

In this example, we used `seaborn` library. `seaborn` is built on top of `matplotlib` and provides a high-level interface for creating attractive and informative statistical graphics. However, we can achive the same by using `matplotlib` library directly.

Let's see the scatter plot of `MedInc` and `HouseAge` features against the target variable `MedHouseVal` using `Matplotlib` library.

In [ ]:
# Scatter plot for 'MedInc' vs 'MedHouseVal'
def plot_feature_target_matplotlib(X, y):
    plt.figure(figsize=(10, 5))

    plt.subplot(1, 2, 1)
    plt.scatter(X["MedInc"], y, alpha=0.5)
    plt.title("Median Income vs Median House Value")
    plt.xlabel("Median Income")
    plt.ylabel("Median House Value")

    # Scatter plot for 'HouseAge' vs 'MedHouseVal'
    plt.subplot(1, 2, 2)
    plt.scatter(X["HouseAge"], y, alpha=0.5)
    plt.title("House Age vs Median House Value")
    plt.xlabel("House Age")
    plt.ylabel("Median House Value")

    plt.tight_layout()
    plt.show()


plot_feature_target_matplotlib(X, y)

### Creating New Features
As we discussed in the previous steps, we want to create a new feature based on the `Latitude` and `Longitude` features. We can create a new feature called `Distance` which in this exaple is the distance of each house from the centre of California with the coordinates (37.16611, -119.44944)`.

> Depending on the domain and data, we can create new features in different ways. For example, we can create a distance of each house from landmark from the ocean as the house price tends to be higher closer to the ocean. However, for simpliciy we'll use the distance from the centre of California.


**Distance Calculation**:

To calculate the distances with high accuracy we need to consider the curvature of the Earth. The _Haversine formula_ is a good choice for calculating the distance between two points on the Earth's surface. However, in this example for simplicity, we'll use the Eucledian distance formula which is good for small distances.

The Eucledian distance between two points $(x_1, y_1)$ and $(x_2, y_2)$ is calculated as:
$$ \text{Distance} = \sqrt{(x_2 - x_1)^2 + (y_2 - y_1)^2} $$




In [ ]:
def scaled_euclidean_distance(lat1, lon1, lat2, lon2):
    # Scale factor, 111 km per degree latitude
    lat_scale = 111

    # km per degree longitude at given latitude
    lon_scale = 111 * np.cos(np.radians(lat1))
    return np.sqrt(
        ((lat2 - lat1) * lat_scale) ** 2 + ((lon2 - lon1) * lon_scale) ** 2
    )

> The scaling factor of `111` comes from the Earth's geometry and is used to approximate the conversion of degrees of latitude or longitude into kilometers.

Now let's apply this formula to calculate the distance of each house from the centre of California.

In [ ]:
# Apply the scaled Euclidean distance

### Feature Scaling
In the final step that we are happy with the selection of feature and added any new features, we want to [normalize](../feature_engineering.md#scaling-and-normalization) the values of these features to have a similar scale. We'll use `StandardScaler` from `sklearn.preprocessing` module to normalize the features. `StandardScaler` uses the [z-score](../feature_engineering.md#z-score-normalization) normalization method to normalize the features.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_normalized = pd.DataFrame(
    scaler.fit_transform(X[selected_features]), columns=selected_features
)

**Should we scale both X and y?**

Typically we normalize `y` in **regression problems** (where the target variable is continuous) or with scale-sensitive models such as Neural Networks.
However, we don't normalize `y` in **classification problems** (where the target variable is categorical) or dealing with scale-insensitive models such as Decision Trees, Random Forests, or Gradient Boosting models.

So, since here we are dealing with a regression problem, we'll normalize the target variable `y` as well.



In [ ]:
y.shape

We need to change the shape of `y` from `(m,)` to `(m, 1)` to be able to use `StandardScaler` on it. In other words, we need to convert `y` from a vector to a matrix with one column and as many rows as needed to accommodate all the data. `reshape(-1, 1)` will change the shape of this array to have as many rows as needed and one column.


In [ ]:
y_normalized = scaler.fit_transform(y.values.reshape(-1, 1))

In [ ]:
plot_features_target_combined_scatter(X_normalized, y_normalized)